<p>
<font size='5' face='Georgia, Arial'>IIC-2233 Apunte Programación Avanzada</font><br>
<font size='1'>&copy; 2015 Karim Pichara - Christian Pieringer. Todos los derechos reservados. Editado por Equipo Docente IIC2233 2018-1 al 2023-2</font>
</p>

# Tabla de contenidos

1. [Introducción](#Introducción)
2. [Procesos y núcleos](#Procesos-y-núcleos)
3. [*Threading*](#Threading)
    1. [Usos de *threads*](#Usos-de-threads)
    2. [Creación de *threads*](#Creación-de-threads)
    3. [`join()`](#join())
    4. [`is_alive()`](#is_alive())
    5. [*Daemons*](#Daemons)
    6. [Timers](#Timers)

## Introducción

Hasta el momento, los programas que hemos escrito solo ejecutan **una secuencia de instrucciones** a la vez. Siguen un único flujo que comienza, ejecuta instrucciones, y en algún momento termina. 

Cada vez que queremos que el usuario ingrese un valor (por ejemplo, una opción en un menú), tenemos que ejecutar `input()` y esperar que el usuario responda. Mientras eso ocurre, nuestro programa debe pausarse mientras espera la respuesta. De la misma manera, si el usuario apreta una tecla y nuestro programa no está esperando que lo haga, no tiene cómo darse cuenta ni cómo recibir esa entrada. Esto es una consecuencia de que los programas que hemos escrito solo siguen un único flujo de instrucciones, y por lo tanto están atentos a sólo una cosa al mismo tiempo.

Muchos programas o aplicaciones que utilizamos realizan múltiples acciones simultáneamente. Cuando escribimos en un procesador de texto, al mismo tiempo se está ejecutando un código que revisa la ortografía. Cuando escribimos código fuente en una IDE, al mismo tiempo también se está ejecutando un código que revisa la sintaxis, que cambia los colores del texto y a veces nos hace sugerencias para completar lo que escribimos. Cuando estamos dentro de un juego, estamos usando teclas para mover a un personaje, pero también están pasando otros eventos en el ambiente del juego al mismo tiempo que movemos el personaje. Si tuviéramos que programar este comportamiento con las herramientas que tenemos, tendríamos que tener un ciclo que, por cada `input()` del usuario, ejecute métodos que revisen ortografía, sintaxis, que hagan sugerencias, que simulen el ambiente del juego, etc. Este comportamiento sería complicado de programar dentro de un programa que pone atención solo a un flujo de instrucciones.

La solución entonces, es que nuestro programa esté atento a múltiples instrucciones simultáneamente. Podemos conseguir esto teniendo "subprogramas": uno que ejecuta el método que revisa ortografía, uno que ejecuta el método que revisa sintaxis, uno que ejecuta el método que hace sugerencias, uno que simula el juego... pero todo dentro del mismo programa. ¿Cómo conseguir esto? El concepto de ***thread*** nos permite definir **múltiples flujos de instrucciones** dentro del mismo código en nuestro programa.


## Procesos y núcleos

En nuestros computadores, podemos ver que se ejecutan varios programas o aplicaciones simultáneamente. Esto lo consigue el computador (en verdad, el sistema operativo) a través de la abstracción de un **proceso**. Un proceso es un **programa en ejecución** que ocupa un espacio de la memoria (RAM) del computador, y ejecuta un flujo de instrucciones. Cuando lanzamos un reproductor de música, un navegador web, un editor de texto, nuestra IDE favorita, o nuestro programa de Python, el computador lo ejecuta mediante un **proceso** que sigue un flujo de instrucciones a partir de la primera línea de código. 

**¿Cómo se ejecutan las instrucciones?**, la unidad que ejecuta instrucciones en un computador es la CPU, la cual posee un núcleo (*core*) que lee una instrucción del programa, la ejecuta, y pasa a la siguiente, hasta que las instrucciones se acaban.

Cuando nuestro programa Python está en ejecución (mediante un proceso), hay un núcleo de nuestro computador que está siguiendo ese flujo de instrucciones. Decimos que hay un proceso que ejecuta nuestro programa Python en la CPU.

Los sistemas modernos poseen múltiples núcleos (*cores*). Es común que nuestros computadores, teléfonos celulares y hasta televisores tengan más de un núcleo, y sean ***dualcore***, ***quadcore***, ***octacore***. Eso significa que puede haber varios procesos, uno en cada núcleo y ejecutando de manera realmente simultánea, o paralela. Cada núcleo ejecuta el flujo de instrucciones de un proceso distinto. Cuando hay más procesos que núcleos para ejecutarlos, los procesos se ponen en una cola, esperando su turno para usar un núcleo. El sistema operativo del computador se encarga de que cada núcleo ejecute algunas instrucciones de un proceso, luego lo deje en pausa, y luego ejecute las instrucciones de otros procesos disponibles. Esto ocurre de manera tan rápida, que lo que observamos es que todos los procesos se ejecutan de manera simultánea dentro del mismo núcleo.

Pero cuando tenemos múltiples flujos de instrucciones en un programa, podemos hacer que varios núcleos ejecuten los distintos flujos de instrucciones **de un mismo proceso** al mismo tiempo. Eso es parte del poder que nos entregan los ***threads***.



## *Threading*

Un ***thread*** (o hilo) es una unidad de ejecución de código dentro de un proceso. Si un proceso es el programa cargado en la memoria del computador, con su código y variables, un ***thread*** lleva la cuenta de en qué parte del código estamos ejecutando. 

![thread](imgs/thread.png)

Todo proceso al crearse posee al menos un ***thread***. A este *thread* le llamamos el *thread principal* del programa. Pero podemos crear más *threads*, y de esa manera hacer que nuestro programa esté ejecutando varias partes del código simultáneamente. Cada ***thread*** lleva su propio registro de variables locales y en qué parte del código del programa se encuentra. Un *thread* también puede crear otros *threads*.

Gracias al sistema operativo, cada *thread* tiene una porción de tiempo para ejecutarse en el núcleo del computador, y podemos ver que varios *threads* se ejecutan de manera simultánea, o *paralela*. La decisión de qué *thread* se ejecuta a continuación y por cuánto tiempo, se conoce como *thread scheduling* o *time slicing*, que funciona –en general– turnando los *threads* de la siguiente manera:

1. Se escoge un *thread* entre los que están disponible para ejecutar.
2. Se ejecuta un cierto número de instrucciones de ese *thread*, o durante un cierto tiempo.
3. Se deja el *thread* actual en espera.
4. Se vuelve al paso 1.

Cuando hay un sólo núcleo en el computador, no existe realmente un paralelismo. Sin embargo, los turnos se pasan de manera tan rápida entre los *threads*, que se comporta como si todos estuvieran avanzando en paralelo. Es por esto que dentro de un sólo núcleo, usar *threads* hace que nuestro programa se ejecute de manera más lenta, pues ahora hay que gastar algo de tiempo en controlar los turnos de los *threads*. Sin embargo, la programación es mucho más clara y sencilla cuando usamos *threads* que se hacen cargo de distintos aspectos de nuestro programa, debido a que no debemos programar de forma explícita el salto entre distintas funcionalidades del programa en un solo hilo de ejecución.

Cuando tenemos sistemas con múltiples núcleos, núcleos distintos podrían estar a cargo de ejecutar *threads* distintos dentro del mismo programa. Nuestro programa entonces podría ejecutar efectivamente de manera más rápida, aprovechando el paralelismo.

### Usos de *threads*

Algunos ejemplos donde resulta útil implementar *threads* son:

- **Separar funciones independientes que trabajan sobre un mismo conjunto de datos pero pueden ejecutarse simultáneamente**. Por ejemplo, un *thread* que se encargue de recibir *input* de texto de un usuario y colocarlo en una cola de texto por procesar, y otro *thread* que procese línea a línea y los saque de la cola. Este tipo de patrón, donde existen procesos que se ejecutan secuencialmente con acceso al mismo conjunto de datos pero que son independientes entre ellos, se conoce como modelo **productor-consumidor** y se ejemplifica en uno de los siguientes *notebooks*.
- **Interfaces gráficas**, donde se necesita interactuar con el usuario mientras se ejecuta algún *thread* de cómputo pesado. Por ejemplo, las interfaces gráficas en un computador: reciben *input* de usuario mediante entradas de texto y botones, mientras otros *threads* actualizan la parte visual simultáneamente.
- **Aplicaciones multiusuario**, en donde cada *thread* se encargaría de las peticiones de cada usuario independientemente. Por ejemplo, un servidor de páginas web debe atender a varios clientes a la vez: cada vez que se conecta un usuario, se debe responder a este de forma correcta, y múltiples usuarios pueden pedir al mismo tiempo una conexión.

### Creación de *threads*

En Python 3, el manejo de *threads* se realiza mediante el módulo `threading`, en donde los *threads* son representados por objetos. 

In [1]:
import threading

La clase `Thread` representa un hilo (o *thread*). La intención es que cada hilo ejecute una secuencia de instrucciones específica de manera simultanea al resto de nuestro programa. Para especificar qué instrucciones ejecutará, se le entrega una función, la que se recibe en el parámetro `target` al crear el *thread*. Es decir, cuando el *thread* comience a funcionar, ejecutará la función entregada en este parámetro.

In [2]:
def contar_diez_ovejas():
    print("Tengo sueño...")
    for numero in range(1, 11):
        print(f"({numero} oveja{'s' if numero > 1 else ''})")
    print("A dormir...")

mi_hilo = threading.Thread(target=contar_diez_ovejas)

Aquí, se creó una instancia de `Thread`, y se almacenó en la variable `mi_hilo`. Recibió como argumento para ejecutar, la función `contar_diez_ovejas`. Notamos que al crear una instancia de `Thread`, este **NO** se ejecuta automáticamente. Es por eso que no vemos *output* generado, ya que no se están ejecutando las instrucciones de `contar_diez_ovejas`, solo definimos un hilo que puede hacerlo.

Para efectivamente ejecutar el *thread*, se debe llamar al método `start()`. **Es altamente recomendable que éste y todos los ejemplos los revises ejecutando código desde una consola, en lugar de los Jupyter Noteboks, para ver como se comporta.**

In [3]:
mi_hilo.start()

Tengo sueño...
(1 oveja)
(2 ovejas)
(3 ovejas)
(4 ovejas)
(5 ovejas)
(6 ovejas)
(7 ovejas)
(8 ovejas)
(9 ovejas)
(10 ovejas)
A dormir...


Es importante notar, que los `Thread` definidos de esta forma son, por defecto, de un solo uso. Es decir, una vez ejecutados, no es posible volver a ejecutarlos. Para eso, es necesario volver a instanciar el *thread*. El siguiente código lanzará un error, ya que el *thread* que intentamos comenzar ya lo hizo antes:

In [4]:
mi_hilo.start()

RuntimeError: threads can only be started once

Como se mencionó inicialmente, todo programa en ejecución tiene al menos un *thread*: el *thread* principal, que corresponde al hilo que ejecuta el flujo principal de nuestro programa. La clase `Thread` admite el argumento `name`, que es un nombre que le podemos dar al *thread* para poder identificarlo. El nombre que se le asigna por defecto al *thread* principal es `"MainThread"`. En el siguiente código, la función `saludar` imprime el nombre del *thread* que lo ejecuta.

In [5]:
def saludar():
    thread_actual = threading.current_thread()
    print(f"Hola desde {thread_actual.name}\n")


hilo_1 = threading.Thread(name="Mi thread 1", target=saludar)
hilo_2 = threading.Thread(name="Mi thread 2", target=saludar)

`threading.current_thread()` es una función que retorna **una referencia de la instancia del *thread* que está ejecutando este código** (el *thread* que está en la CPU). Gracias a eso, se puede acceder a atributos de la instancia de `Thread`, como el nombre (`name`). A continuación, veremos que cada *thread* imprime su nombre correspondiente, incluso el *thread* principal:

In [6]:
# Llamaremos a saludar() desde los threads nombrados
hilo_1.start()
hilo_2.start()
# Ahora llamaremos a saludar() desde el thread principal
saludar()

Hola desde Mi thread 1

Hola desde Mi thread 2

Hola desde MainThread



En el siguiente ejemplo, crearemos tres *threads*: `hilo_lento`, `hilo_rapido_1` e `hilo_rapido_2`, que ejecutarán distintas instancias de las funciones `trabajador_rapido` y `trabajador_lento`.

In [7]:
import threading
import time


def trabajador_rapido():
    # Función rápida, que toma 2 segundos
    thread_actual = threading.current_thread()
    print(f"{thread_actual.name} partiendo...")
    # Pondremos a dormir el thread por 2 segundos simulando
    # que ocurre algun proceso dentro de la función
    time.sleep(2)
    print(f"{thread_actual.name} terminando...")


def trabajador_lento():
    # Función lenta, que toma 6 segundos
    thread_actual = threading.current_thread()
    print(f"{thread_actual.name} partiendo...")
    # Ponemos a dormir el thread por 6 segundos simulando
    # un proceso más largo que el anterior dentro de la función
    time.sleep(6)
    print(f"{thread_actual.name} terminando...")


# Creamos los threads usando la clase Thread
hilo_lento = threading.Thread(name="Hilo lento (6s)", target=trabajador_lento)
hilo_rapido_1 = threading.Thread(name="Hilo rápido (2s)", target=trabajador_rapido)
# Usa el nombre asignado por defecto
hilo_rapido_2 = threading.Thread(target=trabajador_rapido)
print("Thread principal: Fueron creados 3 threads")

Thread principal: Fueron creados 3 threads


In [8]:
# Se inicializan los threads creados
print("Thread principal: Empezaré a iniciar los 3 threads")
hilo_rapido_1.start()  # Dormirá por 2 segundos
hilo_rapido_2.start()  # Dormirá por 2 segundos
hilo_lento.start()  # Dormirá por 6 segundos
print("Thread principal: Fueron iniciados 3 threads")
# Todas estas líneas que siguen serán ejecutadas mientras los threads
# se ejecutan independientemente del programa principal

print()
# El thread principal ejecutará lo que queda de código
# mientras los otros 3 threads hacen lo suyo

for i in range(10):
    print(f"Thread principal: Segundo actual: {i}")
    time.sleep(1)

Thread principal: Empezaré a iniciar los 3 threads
Hilo rápido (2s) partiendo...
Thread-6 (trabajador_rapido) partiendo...
Hilo lento (6s) partiendo...
Thread principal: Fueron iniciados 3 threads

Thread principal: Segundo actual: 0
Thread principal: Segundo actual: 1
Hilo rápido (2s) terminando...Thread-6 (trabajador_rapido) terminando...

Thread principal: Segundo actual: 2
Thread principal: Segundo actual: 3
Thread principal: Segundo actual: 4
Thread principal: Segundo actual: 5
Hilo lento (6s) terminando...
Thread principal: Segundo actual: 6
Thread principal: Segundo actual: 7
Thread principal: Segundo actual: 8
Thread principal: Segundo actual: 9


#### ¿Qué pasa con los `print`?

A partir de este ejemplo en adelante, es probable que al ejecutar `print` falten o sobran saltos de línea. La pregunta es, ¿tienen que ver los *threads* con este extraño comportamiento? La respuesta es sí.

De acuerdo a [este post en StackOverflow](https://stackoverflow.com/questions/31142566/print-skipping-newline) esto se produce porque – a nivel de máquina – escribir el texto del `print` y escribir el salto de línea son **dos instrucciones distintas**. Por lo tanto, es posible que un *thread* imprima el texto, se pause ese *thread*, luego otro *thread* imprima su texto y su respectivo salto de línea, y luego al volver al primer *thread* este imprima el salto de línea que faltaba.



**Pon en práctica la creación de *threads* simples realizando el ejercicio propuesto 1 del archivo 1.**

En el ejemplo, vemos que una vez inicializados los *threads*, **el programa principal continúa con el resto de instrucciones mientras los *threads* se ejecutan**. Luego los tres *threads* terminan independientemente en distintos momentos. El programa principal esperará a que todos los *threads* hayan terminado correctamente.

También es posible, mediante el atributo `args` o `kwargs`, pasar los argumentos a la función definida como `target` del *thread*.

In [9]:
import threading
import time


def contar_ovejas_hasta(max_ovejas):
    thread_actual = threading.current_thread()
    print(f"{thread_actual.name} tiene sueño...")
    for numero in range(1, max_ovejas + 1):
        time.sleep(1)
        print(f"({thread_actual.name}: {numero} oveja{'s' if numero > 1 else ''})")
    print(f"{thread_actual.name} a dormir...")


# Se crean los threads usando la clase Thread, asociada a la función objetivo para 
# ser ejecutada por el thread, y los atributos de la función son ingresados en 
# args o kwargs

t1 = threading.Thread(name="Thread 1", target=contar_ovejas_hasta, args=(10,))
t2 = threading.Thread(name="Thread 2", target=contar_ovejas_hasta, kwargs={"max_ovejas": 15})
t1.start()
t2.start()

Thread 1 tiene sueño...
Thread 2 tiene sueño...
(Thread 1: 1 oveja)(Thread 2: 1 oveja)

(Thread 2: 2 ovejas)(Thread 1: 2 ovejas)

(Thread 2: 3 ovejas)(Thread 1: 3 ovejas)

(Thread 2: 4 ovejas)(Thread 1: 4 ovejas)

(Thread 1: 5 ovejas)(Thread 2: 5 ovejas)

(Thread 2: 6 ovejas)(Thread 1: 6 ovejas)

(Thread 2: 7 ovejas)(Thread 1: 7 ovejas)

(Thread 1: 8 ovejas)(Thread 2: 8 ovejas)

(Thread 1: 9 ovejas)(Thread 2: 9 ovejas)

(Thread 1: 10 ovejas)(Thread 2: 10 ovejas)

Thread 1 a dormir...
(Thread 2: 11 ovejas)
(Thread 2: 12 ovejas)
(Thread 2: 13 ovejas)
(Thread 2: 14 ovejas)
(Thread 2: 15 ovejas)
Thread 2 a dormir...


**Pon en práctica la creación de *threads* con argumentos realizando el ejercicio propuesto 2 del archivo 1.

En concordancia con el modelamiento basado en OOP, podemos crear también *threads* como instancias de una clase que hereda desde la clase `Thread`. Es decir, podemos crear clases para *threads* con comportamiento común. Para definir las instrucciones que serán ejecutadas por la instancia de *thread*, se debe hacer *override* del método `run()` en la clase, el cual se ejecuta luego de llamar a `start`, al igual que antes. Con esto, ahora podremos personalizar este método para definir las instrucciones a ejecutar directamente para cada clase de *thread*.

In [10]:
import threading
import time


class CuentaOvejas(threading.Thread): # Hereda de Thread
    """Este será nuestro nuevo Cuenta Ovejas basado en Thread"""
    
    def __init__(self, nombre, max_ovejas):
        # En el caso de los threads, lo primero es invocar al init original. SIEMPRE.
        super().__init__(name=nombre)
        self.max_ovejas = max_ovejas # Se agrega un atributo de instancia extra
    
    def run(self):
        # Este metodo define las instrucciones a ejecutar de este thread
        # cuando ejecutamos el metodo start()
        print(f"{self.name} tiene sueño...")
        tiempo_partida = time.time()
        for numero in range(1, self.max_ovejas + 1):
            time.sleep(1)
            print(f"({self.name}: {numero} oveja{'s' if numero > 1 else ''})")
        print(f"{self.name} a dormir...")
        print(f"{self.name} se durmió después de {time.time() - tiempo_partida} seg.")

        
class CuentaLiebres(threading.Thread): # Hereda de Thread
    """
    Este será un nuevo Cuenta Liebres basado en Thread
    Las liebres son más rápidas, así que cuenta dos por segundo
    """
    
    def __init__(self, nombre, max_liebres):
        super().__init__(name=nombre)
        self.max_liebres = max_liebres
    
    def run(self):
        print(f"{self.name} tiene sueño...")
        tiempo_partida = time.time()
        for numero in range(1, self.max_liebres + 1):
            if numero % 2 == 1:
                time.sleep(1)
            print(f"({self.name}: {numero} liebre{'s' if numero > 1 else ''})")
        print(f"{self.name} a dormir...")
        print(f"{self.name} se durmió después de {time.time() - tiempo_partida} seg.")
        

# Se crean los threads
cuenta_ovejas = CuentaOvejas("Hernán", 10)
cuenta_liebres = CuentaLiebres("Daniela", 10)
print("Thread principal: Fueron creados 2 threads")

Thread principal: Fueron creados 2 threads


In [11]:
# Se inicializan los threads creados
cuenta_ovejas.start()
cuenta_liebres.start()
print("Thread principal: Fueron iniciados 2 threads")
# Todas estas líneas serán ejecutadas mientras los threads se ejecutan
# independientemente del programa principal
    
print()
# El thread principal ejecutará lo que queda código
# mientras los otros 2 threads hacen lo suyo
for i in range(10):
    print(f"Thread principal: Segundo actual: {i}")
    time.sleep(1)

Hernán tiene sueño...
Daniela tiene sueño...
Thread principal: Fueron iniciados 2 threads

Thread principal: Segundo actual: 0
Thread principal: Segundo actual: 1(Daniela: 1 liebre)
(Daniela: 2 liebres)
(Hernán: 1 oveja)

(Daniela: 3 liebres)
(Daniela: 4 liebres)
(Hernán: 2 ovejas)
Thread principal: Segundo actual: 2
Thread principal: Segundo actual: 3(Hernán: 3 ovejas)
(Daniela: 5 liebres)
(Daniela: 6 liebres)

(Daniela: 7 liebres)(Hernán: 4 ovejas)

(Daniela: 8 liebres)
Thread principal: Segundo actual: 4
(Daniela: 9 liebres)(Hernán: 5 ovejas)
Thread principal: Segundo actual: 5

(Daniela: 10 liebres)
Daniela a dormir...
Daniela se durmió después de 5.0398828983306885 seg.
(Hernán: 6 ovejas)Thread principal: Segundo actual: 6

Thread principal: Segundo actual: 7(Hernán: 7 ovejas)

(Hernán: 8 ovejas)Thread principal: Segundo actual: 8

(Hernán: 9 ovejas)Thread principal: Segundo actual: 9

(Hernán: 10 ovejas)
Hernán a dormir...
Hernán se durmió después de 10.106433629989624 seg.


### `join()`

Hasta ahora hemos visto comportamiento de *threads* que se ejecutan "en paralelo" y de forma independiente al flujo del programa principal. Pero hay ocasiones donde el programa principal puede necesitar que otros *threads* terminen antes de ejecutar ciertas instrucciones.

Por ejemplo, un programa que debe obtener el total de compras de una tienda, donde el detalle de las compras se encuentran en cinco archivos en distintos formatos. El programa delega a cinco *threads* distintos el leer y sumar el total de compras de cada archivo, **espera** a que cada *thread* termine de hacer sus cálculos y luego obtiene los cinco resultados y los suma.

Si necesitamos que el programa principal espere a que termine la ejecución de algún *thread* (o grupo de *threads*), debemos usar el método `join(timeout=None)` después de que el *thread* ha sido iniciado mediante `start()`.

Al usar `join()`, **el *thread* que llama al método** queda bloqueado hasta que los *threads* referenciados terminen correctamente. No obstante, si se especifica un `timeout` (en segundos) el programa esperará al *thread* referenciado solo por ese tiempo y luego continuará.

La figura siguiente explica gráficamente la ejecución de un programa usando `join()`.

![join](imgs/join.png)

En el código esto se refleja como que *Main Thread* incluye las llamadas `T1.join()` y `T2.join()`.  Como es *Main Thread* quien llama a `join`, es ese el *thread* que se bloquea esperando a otro; primero espera a `T1` y luego a `T2`. Por otro lado, no sólo el *Main Thread* puede esperar: **cualquier *thread* puede esperar a cualquier otro** usando `join()`.

Veamos ahora una extensión del ejemplo anterior pero con la incorporación de `join()`. Aquí, los ayudantes del curso esperan a que los profesores (o casi todos) se duerman para empezar a festejar. Notar que el comportamiento de los ayudantes está en el programa principal, y el festejo no comienza hasta después de que la mayoría de los *threads* ejecutados terminan.

In [12]:
import threading
import time


# Usamos la definicion de los Thread declarados en el ejemplo anterior
# Se crean los threads usando la clase Thread.
hernan = CuentaOvejas("Hernán", 5)
daniela = CuentaOvejas("Daniela", 7)
fran = CuentaLiebres("Fran", 5)
joaquin = CuentaLiebres("Joaquín", 6)
paqui = CuentaLiebres("Paqui", 20)

# Se inicializan los threads creados
hernan.start()
daniela.start()
fran.start()
joaquin.start()
paqui.start()
print("Ayudantes: Los profes se fueron a dormir...")

# Aquí incorporamos el método join() para bloquear el programa principal
daniela.join()  # No especificamos timeout, esperará lo que sea necesario
print("Ayudantes: ¡DANIELA SE DURMIÓ!")
hernan.join() # No especificamos timeout, esperará lo que sea necesario
print("Ayudantes: ¡HERNÁN SE DURMIÓ!")
fran.join() # No especificamos timeout, esperará lo que sea necesario
print("Ayudantes: ¡FRAN SE DURMIÓ!")
joaquin.join() # No especificamos timeout, esperará lo que sea necesario
print("Ayudantes: ¡JOAQUÍN SE DURMIÓ!")
paqui.join(1)  # Esperaremos máximo 1 segundo después del último dormido, ya es muy tarde
print("Ayudantes: ¡(casi todos) los profes se durmieron! ¡A festejar!")

# En este punto, el programa ha esperado por los cuatro threads que creamos
# Estas líneas serán ejecutadas después de que los threads hayan terminado
for _ in range(6):
    print("Ayudantes: 🎵🎶🎵🎶🎵🎶🎵🎶🎵🎶🎵🎶🎵🎶🎵🎶")
    time.sleep(1)
print("Ayudantes: Ojalá no nos hayan escuchado...")

Hernán tiene sueño...
Daniela tiene sueño...
Fran tiene sueño...
Joaquín tiene sueño...
Paqui tiene sueño...
Ayudantes: Los profes se fueron a dormir...
(Daniela: 1 oveja)(Joaquín: 1 liebre)
(Joaquín: 2 liebres)
(Fran: 1 liebre)
(Fran: 2 liebres)

(Hernán: 1 oveja)
(Paqui: 1 liebre)
(Paqui: 2 liebres)
(Fran: 3 liebres)(Joaquín: 3 liebres)
(Joaquín: 4 liebres)
(Hernán: 2 ovejas)

(Fran: 4 liebres)
(Paqui: 3 liebres)
(Paqui: 4 liebres)
(Daniela: 2 ovejas)
(Daniela: 3 ovejas)(Hernán: 3 ovejas)
(Fran: 5 liebres)
Fran a dormir...
Fran se durmió después de 3.037274122238159 seg.

(Joaquín: 5 liebres)
(Joaquín: 6 liebres)
Joaquín a dormir...
Joaquín se durmió después de 3.0382707118988037 seg.
(Paqui: 5 liebres)
(Paqui: 6 liebres)
(Daniela: 4 ovejas)(Hernán: 4 ovejas)
(Paqui: 7 liebres)
(Paqui: 8 liebres)

(Daniela: 5 ovejas)(Paqui: 9 liebres)
(Paqui: 10 liebres)
(Hernán: 5 ovejas)
Hernán a dormir...
Hernán se durmió después de 5.062022924423218 seg.

(Daniela: 6 ovejas)(Paqui: 11 liebres)
(P

🤔 **¿Qué ocurre si tenemos lo siguiente?**

```python
thread_1.start()
thread_1.join()
thread_2.start()
thread_2.join()
thread_3.start()
thread_3.join()
```

¿Tendremos multiples *threads* ejecutandose? 🤔 🤔

La respuesta es **no**. En este código, se está empezando un *thread* y se esta esperando que finalice (`.join()`) antes de continuar con la siguiente linea de código que se encarga de empezar el siguiente *thread*. Cuando trabajos con *threads* hay que tener mucho cuidado en la ubicación de los `.start()` y los `.join()`. El comportamiento estándar sería empezar todos los *threads* y luego comenzar con los `join()` para esperarlos.

**Ahora intenta realizar el ejercicio propuesto 3 del archivo 1 para fijar un orden de ejecución de *threads*.**

### `is_alive()`

Es posible identificar si un *thread* todavía está en funcionamiento mediante el uso del método `is_alive()`. Por lo general este método se implementa para saber el estado del *thread* después del uso de `join()`, al cual se le ha definido su tiempo máximo de espera.

In [13]:
import threading
import time


# Usamos la definicion de los Thread declarados en el ejemplo anterior
# Se crean los threads usando la clase Thread.
hernan = CuentaOvejas("Hernán", 6)
daniela = CuentaOvejas("Daniela", 3)
fran = CuentaLiebres("Fran", 6)
joaquin = CuentaLiebres("Joaquín", 8)
paqui = CuentaLiebres("Paqui", 15)

# Se inicializan los threads creados
hernan.start()
daniela.start()
fran.start()
joaquin.start()
paqui.start()
print("Ayudantes: Los profes se fueron a la cama...")

daniela.join()  # Esperaremos lo que sea necesario.
print("Ayudantes: ¡DANIELA SE DURMIÓ!")
hernan.join() # No especificamos timeout, esperará lo que sea necesario
print("Ayudantes: ¡HERNÁN SE DURMIÓ!")
fran.join() # Esperaremos lo que sea necesario.
print("Ayudantes: ¡FRAN SE DURMIÓ!")
joaquin.join() # Esperaremos lo que sea necesario.
print("Ayudantes: ¡JOAQUÍN SE DURMIÓ!")
paqui.join(1)  # Esperaremos máximo 1 segundos después del último dormido, ya es muy tarde

if paqui.is_alive():
    print("Ayudantes: Paqui sigue despierta 😞. A la casa cabros.")
else:
    print("Ayudantes: ¡Todos los profes se durmieron! ¡A festejar!")
    for i in range(6):
        print("Ayudantes: 🎵🎶🎵🎶🎵🎶🎵🎶🎵🎶🎵🎶🎵🎶🎵🎶")
        time.sleep(1)

Hernán tiene sueño...
Daniela tiene sueño...
Fran tiene sueño...
Joaquín tiene sueño...
Paqui tiene sueño...
Ayudantes: Los profes se fueron a la cama...
(Joaquín: 1 liebre)
(Joaquín: 2 liebres)
(Daniela: 1 oveja)
(Hernán: 1 oveja)
(Paqui: 1 liebre)
(Paqui: 2 liebres)
(Fran: 1 liebre)
(Fran: 2 liebres)
(Joaquín: 3 liebres)(Daniela: 2 ovejas)
(Hernán: 2 ovejas)

(Joaquín: 4 liebres)
(Paqui: 3 liebres)
(Paqui: 4 liebres)
(Fran: 3 liebres)
(Fran: 4 liebres)
(Joaquín: 5 liebres)(Paqui: 5 liebres)
(Paqui: 6 liebres)

(Joaquín: 6 liebres)
(Fran: 5 liebres)
(Fran: 6 liebres)
Fran a dormir...
Fran se durmió después de 3.050499677658081 seg.
(Daniela: 3 ovejas)
Daniela a dormir...
Daniela se durmió después de 3.050499677658081 seg.
Ayudantes: ¡DANIELA SE DURMIÓ!
(Hernán: 3 ovejas)
(Joaquín: 7 liebres)(Hernán: 4 ovejas)

(Joaquín: 8 liebres)
Joaquín a dormir...
Joaquín se durmió después de 4.070190191268921 seg.
(Paqui: 7 liebres)
(Paqui: 8 liebres)
(Hernán: 5 ovejas)(Paqui: 9 liebres)
(Paqui: 1

In [14]:
print("(Ayudantes vuelven)")
print("Ayudantes: Esperemos un poco más cabros...")
time.sleep(2)
for profe in [hernan, daniela, fran, joaquin, paqui]:
    if profe.is_alive():
        print(f"Ayudantes: {profe.name} aún está despiert@ 😞")
    else:
        print(f"Ayudantes: ¡{profe.name} se quedó dormid@!")

(Ayudantes vuelven)
Ayudantes: Esperemos un poco más cabros...
Ayudantes: ¡Hernán se quedó dormid@!
Ayudantes: ¡Daniela se quedó dormid@!
Ayudantes: ¡Fran se quedó dormid@!
Ayudantes: ¡Joaquín se quedó dormid@!
Ayudantes: ¡Paqui se quedó dormid@!


### *Daemons*

En los ejemplos anteriores, el programa espera a que todos los *threads* terminen su trabajo antes de terminar su ejecución. Los ***daemon threads*** son aquellos que, **a pesar de que estén aún corriendo, no impiden que el programa principal termine**.

Si no existieran los *daemon threads*, tendríamos que preocuparnos de tener en cuenta todos nuestros *threads* y preocuparnos de terminarlos a todos para que el programa principal pueda terminar. Con los *daemon threads*, podemos ponerlos a correr y olvidarnos de ellos, y cuando el programa termina, los *daemon threads* serán terminados inmediatamente. El programa principal terminará cuando todos los *threads no-daemon* hayan finalizado.

Para especificar que un *thread* debe ser *daemon*, se especifica el parámetro `daemon=True` en el constructor del *thread*. Una forma alternativa es escribir el atributo `daemon` del *thread*, después de haberlo creado.

#### *Warning*: este notebook no refleja el efecto de *daemon*.

Debido al funcionamiento interno de los Jupyter Notebooks, no se puede notar el efecto de utilizar esta propiedad de los *threads* dentro en las celdas del *notebook*. Es por esto que el siguiente código se encuentra en el archivo adjunto `daemon_threads.py`, ubicado en la carpeta `scripts`, para ser ejecutado **fuera de este *notebook*, desde una consola**.


```python
import threading
import time


def dormilon():
    print(f"{threading.current_thread().name} tiene sueño...")
    time.sleep(2)
    print(f"{threading.current_thread().name} se durmió.")

    
def con_insomnio():
    print(f"{threading.current_thread().name} tiene sueño...")
    time.sleep(10)
    print(f"{threading.current_thread().name} se durmió.")


# Forma 1 de hacer un thread daemon
dormilon = threading.Thread(name="Dormilón", target=dormilon, daemon=True)
# Forma 2 de hacer un thread daemon
con_insomnio = threading.Thread(name="Con insomnio", target=con_insomnio)
con_insomnio.daemon = True

# Se inicializan los threads
dormilon.start()
con_insomnio.start()

```

La siguiente figura muestra la salida en la consola para el programa **con** los *threads* configurados como *daemon*. Los *threads* se interrumpen abruptamente, luego de que el programa principal termina su ejecución.

![daemon](imgs/daemon.png)

Comparemos el resultado anterior con la salida en la consola para el programa **sin** los *threads* configurados como *daemon* (`daemon=False`), como muestra la imagen siguiente. Podemos ver que la ejecución de los *threads* fue completada y que el programa no se cierra hasta que los *threads* terminan.

![non_daemon](imgs/non-daemon.png)

En la penúltima imagen, podemos apreciar que el programa principal termina antes que los *daemon threads*. Es posible hacer que el programa espere a un *daemon thread* mediante el uso de `join()`, lo que tendría sentido cuando debemos esperar algún resultado del *thread* para el resto del programa.

In [15]:
import threading
import time


def dormilon():
    print(f"{threading.current_thread().name} tiene sueño...")
    time.sleep(2)
    print(f"{threading.current_thread().name} se durmió.")

    
def con_insomnio():
    print(f"{threading.current_thread().name} tiene sueño...")
    time.sleep(10)
    print(f"{threading.current_thread().name} se durmió.")


# Forma 1 de hacer un thread daemon
dormilon = threading.Thread(name="Dormilón", target=dormilon, daemon=True)
# Forma 2 de hacer un thread daemon
con_insomnio = threading.Thread(name="Con insomnio", target=con_insomnio)
con_insomnio.daemon = True

# Se inicializan los threads
dormilon.start()
con_insomnio.start()

# Esperamos los threads.
# Lo esperamos por una cantidad indefinida de tiempo
dormilon.join()
# Esperamos sólo 5 segundos
con_insomnio.join(5)

Dormilón tiene sueño...
Con insomnio tiene sueño...
Dormilón se durmió.
Con insomnio se durmió.


Un aspecto importante a notar es que una vez que se ha llamado el método `start()` un *thread* no puede cambiar desde *daemon* a *no-daemon*, o vicecersa. Si tratamos de hacerlo, se lanza una excepción del tipo `RuntimeError`.

In [16]:
dormilon = threading.Thread(name="Dormilón", target=dormilon, daemon=True)
dormilon.start()

# Noo!
dormilon.daemon = False

dormilon.join()

Exception in thread Dormilón:
Traceback (most recent call last):
  File "C:\Users\franc\AppData\Local\Programs\Python\Python310\lib\threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "C:\Users\franc\AppData\Local\Programs\Python\Python310\lib\threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
TypeError: 'Thread' object is not callable


RuntimeError: cannot set daemon status of active thread

Al hacer una subclase de `Thread`, que queremos que sea *daemon*, podemos indicar que el *thread* es *daemon* durante su inicialización, usando el atributo `daemon`.

In [17]:
class Daemon(threading.Thread):
    
    def __init__(self):
        super().__init__()
        # Cuando inicializamos el thread lo declaramos como daemon
        self.daemon = True
    
    def run(self):
        print("Daemon thread: Empezando...")
        time.sleep(2)
        print("Daemon thread: Terminando...")

        
daemon = Daemon()
daemon.start()
daemon.join()

Daemon thread: Empezando...
Daemon thread: Terminando...


Ahora que ya entendemos que un thread daemon se puede interrumpir su ejecución si el programa principal se acaba. Vamos a revisar el siguiente código que está en `daemon_threads_print_error`, dentro de la carpeta `scripts`.

```python
from threading import Thread
import time

def presentar(i): 
    print("Hola soy el thread {}".format(i, i))

def main():
    # Crear threads
    threads = []
    for i in range(25):
        threads.append(Thread(target=presentar, daemon=True, args=(i+1,)))

    # Empezar los threads
    for thread in threads:
        thread.start()

if __name__ == "__main__":
    print("Empezar programa, crearé 25 threads")
    main()
```

En este código, se crean 25 *thread daemon* los cuales se van a presentar y se acaba todo. Si ejecutamos este archivo en consola, un posible resultado es:


![daemon_error](imgs/daemon_error.png)

La última parte da miedo... ¿que es eso de `Fatal Python error`? ¿por qué no vemos el `print` del **thread 23**?

Este error se debe a que un *thread* estaba junto en proceso de imprimir el stirng en consola cuando fue interrumpido por el programa principal. Dado que imprimir en consola implica enviarle al sistema operativo una acción, se genera este error porque fue gatillada una interrumpición al sistema operativo por parte del hilo princiapl. Este error es normal que ocurra si es que estamos dejando que algún *daemon thread* maneje `prints` y luego cortamos el programa principal de forma abrupta.

### *Timers*

La clase `Timer` es una subclase de la clase `Thread` y permite ejecutar un proceso o acción después que ha pasado un determinado tiempo. Es decir, al ser ejecutado, espera el tiempo que se le especifica y luego ejecuta las instrucciones determinadas. `Timer` requiere, como parámetros básicos, el tiempo en segundos después del cual debe ejecutarse el *thread*, la función a ejecutar, y los argumentos de la función a ejecutar.

El método `cancel()` permite, si se requiere, cancelar la ejecución del *timer* antes que este sea ejecutado.

#### *Warning*: este notebook podría no refleja el efecto de un *Timer*. Recomendamos crear un archivo .py con el código a continuación y ejecutarlo para ver el efecto esperado.

Según la versión de jupyter notebook que dispongan, puede que no se muestre el efecto de utilizar esta propiedad de los *threads* en las celdas del notebook, es decir, que los *threads* no esperen el tiempo necesario antes de ejecutarse. En caso que ocurra eso, recomendamos probar el código de la siguiente celda en una consola y ejecutar desde ahí.


In [18]:
def mi_timer(ruta_archivo):
    with open(ruta_archivo) as archivo:
        for linea in archivo:
            print(linea.strip())

t1 = threading.Timer(4.0, mi_timer, args=("files/mensaje_01.txt",))
t2 = threading.Timer(1.0, mi_timer, kwargs={"ruta_archivo": "files/mensaje_02.txt"})

print("Activando timer 1 que comenzará luego de 4 segundos")
t1.start() # el thread t1 comenzará después de 4 seconds

print("Activando timer 2 que comenzará luego de 1 segundos")
t2.start() # el thread t2 comenzará después de 1 seconds

Activando timer 1 que comenzará luego de 4 segundos
Activando timer 2 que comenzará luego de 1 segundos
Hola :) acabas de ejecutar el timer 2
Â¡Hola!, acabas de ejecutar el timer 1
